### Steps to Solve 'Scenarios' Linear Optimization Problem:
1) Ensure functions in optimization problem are indeed linear (scalar only)
2) What is the "Objective Function"? It is to minimize costs subject to our constraints
3) What are the "Constraints"? Clear return hurdles in each scenario with position size >= 0 (long-only!)
4) What are the "Decision Variables" (solution variables)? Asset position sizes (x values in each scenario)
5) Define the "Problem" (using cvx.Problem)
6) Solve the "Problem" (using cvx.Solve)
7) Print the Value of the Problem which in below example is optimized min cost (using print) 
8) Print the Value of the Solutions which in below example are asset position sizes (using print). ROUND UP!

### Creates three new variables and prints them using data from below table:
|        | Equities | Fixed Income | Public Alternative | Private Alternative | Req
| :----- | :------:| :----: | :-----: | :-----: | :--:
| Cost        | 2.0 | 3.0 | 1.0 | 0.5
| Scenario 1  | 0.2 | 1.0 | 0.1 | 0.5 | 10
| Scenario 2  | 0.5 | 1.2 | 1.0 | 0.8 | 20
| Scenario 3  | 1.0 | 0.2 | 1.3 | 1.2 | 15
##### 1) S --> Array of the return values for the assets 
     Linear Algebra: 3x4 (coefficient) matrix
     Optimization Problem: Input for the "Objective Function"
##### 2) c --> Array of the cost values for each asset 
     Linear Algebra: Row vector
     Optimization Problem: Input for the "Objective Function" 
##### 3) r --> Array of the scenario return requirement hurdles 
     Linear Algebra: Column vector
     Optimization Problem: Inputs for "Constraints" on Objective Function

In [1]:
import numpy as np
import cvxpy as cvx #for convex optimization problems (transforms problem to standard form, calls solver, & unpacks result)

In [2]:
S = np.array([
    [0.2, 1.0, 0.1, 0.5],
    [0.5, 1.2, 1.0, 0.8],
    [1.0, 0.2, 1.3, 1.2]
    ])
c = np.array([2,3,1,5])
r = np.array([10,20,15])

print('Asset Returns Matrix (Constraint Input): \n\n'+str(S)+
      '\n\nCost Row Vector (Objective Input): '+str(c)+
      '\n\nReturn Col Vector Hurdle (Constraint Input): '+str(r))

Asset Returns Matrix (Constraint Input): 

[[0.2 1.  0.1 0.5]
 [0.5 1.2 1.  0.8]
 [1.  0.2 1.3 1.2]]

Cost Row Vector (Objective Input): [2 3 1 5]

Return Col Vector Hurdle (Constraint Input): [10 20 15]


In [3]:
#np.shape([array_name]) returns tuple (x, y) of array shape where x, y is # rows, cols and prints
#Similar to how matrix is defined in linear algebra
#Below we set tuple equal to two distinct variables 'm' (scenarios) & 'd' (assets) in a list
[m,d] = np.shape(S)
print('Row #: '+str(m)+' | Col #: '+str(d))

Row #: 3 | Col #: 4


### Creates x, obj, constraints variables using cvs to setup and solve optimization problem:
##### 1) x: "Decision Variable"
"Decision Variable" in optimization (asset position size here) and set by cvx.Variable(int) where the int (integer) parameter/argument is the number of (decision) variable(s) so here it is '4' b/c there are 4 assets/cols
#### 2) obj: "Objective Function"
Minimizing cost here and set by cvx.Minimize(), other functions like "Maximize" can be used) where cvx function populated w/ array or some operation of multiple arrays. Here we multiply row vector 'c' (cost) by column vector 'x' (asset variables) and request output minimized 
#### 3) constraints: Constrains the Objective Function (you can dump all constraints into a list here)
Here we take product of 'S' and 'x' (asset return matrix * asset position size col vector) and ensure it's >= 'r' (return hurdle col vector) while also enforcing that 'x' >= 0 (no shorts: we need position size to be zero or positive)  

In [4]:
x = cvx.Variable(d)
obj = cvx.Minimize(c@x)
constraints = [S@x >= r, x >=0]

In [5]:
#cvx.Problem creates object representing optimization problem: takes objective & constraint as args (obj MUST BE min/max)
prob = cvx.Problem(obj,constraints)
#prob.solve() (or cvx.Problem object variable name .solve()) solves optimization problem and prints solution value 
prob.solve()

#Try/except is so we can return nature of Problem Status if we get error on 'try' prints
try:
    #Value of optimization problem (value of objective function at optimal solution, so here it's minimized cost value)
    print('Value of Optimization Problem: ' + str(round(prob.value, 2)))
    #Prints 'x' value ("Decision Variables" --> here position size). NEED print(x.value) syntax to work properly!!!
    print('Value of Solutions (Asset Position Sizes): ' + str(np.round(x.value, 2)))
    print('Probem Status: ' + str(prob.status)) 
except:
   #3 statuses of optimization problem: Infeasible: no solution given constraints | Optimal: good | Unbounded: 
    #constraints not defined well enough (+/- infinity value). ARBITRAGE EXCEPTION (Unounded implies infinite profit)
    print('\nProbem Status: ' + str(prob.status)) 

Value of Optimization Problem: 37.11
Value of Solutions (Asset Position Sizes): [ 0.    8.98 10.16  0.  ]
Probem Status: optimal


Rerunning solution to allow shorts. CREATES UNBOUNDED PROBLEM

In [6]:
#Rerun after loosening constraints to expand feasible solutions to problem by allowing shorts (negative 'x' values)
#Generally when EXPANDING feasible solution set optimization value goes lower
x_short = cvx.Variable(d)
obj_short = cvx.Minimize(c@x_short)
constraints_short = [S@x_short >= r]

prob_short = cvx.Problem(obj_short,constraints_short)
prob_short.solve()

try:
    print('Value of Optimization Problem w/ Shorting: ' + str(prob_short.value))
    print('Value of Solutions w/ Shorting (Asset Position Sizes): ' + str(np.round(x_short.value, 2)))
    print('Probem Status: ' + str(prob_short.status)) 
except:
    print('\nProbem Status: ' + str(prob_short.status))

Value of Optimization Problem w/ Shorting: -inf

Probem Status: unbounded
